<a href="https://colab.research.google.com/github/Fadelhamkaa/ML_Naive-Bayes_Manual-/blob/main/LP4_Muhammad_Fadel_Hamka_60200122116.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_excel('/content/Lokasi Perumahan.xlsx')
print("Dataset:")
print(df)

Dataset:
  Harga Tanah Jarak dai pusat kota Ada angkutan umum Dipilih untuk perumahan
0       Murah                Dekat             Tidak                     Iya
1      Sedang                Dekat             Tidak                     Iya
2       Mahal                Dekat             Tidak                     Iya
3       Mahal                 Jauh             Tidak                   Tidak
4       Mahal               Sedang             Tidak                   Tidak
5      Sedang                 Jauh               Ada                   Tidak
6       Murah                 Jauh               Ada                   Tidak
7       Murah               Sedang             Tidak                     Iya
8       Mahal                 Jauh               Ada                   Tidak
9      Sedang               Sedang               Ada                     Iya


In [ ]:
# Encode fitur kategorikal ke nilai numerik
le = LabelEncoder()
encoded_df = df.copy()

for col in encoded_df.columns:
    encoded_df[col] = le.fit_transform(encoded_df[col])

print("Dataset setelah encoding:")
print(encoded_df)

Dataset setelah encoding:
   Harga Tanah  Jarak dai pusat kota  Ada angkutan umum  \
0            1                     0                  1   
1            2                     0                  1   
2            0                     0                  1   
3            0                     1                  1   
4            0                     2                  1   
5            2                     1                  0   
6            1                     1                  0   
7            1                     2                  1   
8            0                     1                  0   
9            2                     2                  0   

   Dipilih untuk perumahan  
0                        0  
1                        0  
2                        0  
3                        1  
4                        1  
5                        1  
6                        1  
7                        0  
8                        1  
9                        0  


In [ ]:
# Menghitung probabilitas prior
label_counts = encoded_df['Dipilih untuk perumahan'].value_counts()
total_samples = len(encoded_df)
prior_probs = label_counts / total_samples

# Menghitung probabilitas likelihood
feature_probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
features = ['Harga Tanah', 'Jarak dai pusat kota', 'Ada angkutan umum']
target = 'Dipilih untuk perumahan'

for feature in features:
    for value in encoded_df[feature].unique():
        for class_value in encoded_df[target].unique():
            num = len(encoded_df[(encoded_df[feature] == value) & (encoded_df[target] == class_value)])
            den = label_counts[class_value]
            feature_probs[feature][value][class_value] = num / den if den != 0 else 0

print("\nProbabilitas Prior:")
print(prior_probs)

print("\nProbabilitas Likelihood:")
for feature in features:
    print(f"\n{feature}:")
    for value in feature_probs[feature]:
        for class_value in feature_probs[feature][value]:
            print(f"P({feature}={value} | Kelas={class_value}) = {feature_probs[feature][value][class_value]:.4f}")


Probabilitas Prior:
Dipilih untuk perumahan
0    0.5
1    0.5
Name: count, dtype: float64

Probabilitas Likelihood:

Harga Tanah:
P(Harga Tanah=1 | Kelas=0) = 0.4000
P(Harga Tanah=1 | Kelas=1) = 0.2000
P(Harga Tanah=2 | Kelas=0) = 0.4000
P(Harga Tanah=2 | Kelas=1) = 0.2000
P(Harga Tanah=0 | Kelas=0) = 0.2000
P(Harga Tanah=0 | Kelas=1) = 0.6000

Jarak dai pusat kota:
P(Jarak dai pusat kota=0 | Kelas=0) = 0.6000
P(Jarak dai pusat kota=0 | Kelas=1) = 0.0000
P(Jarak dai pusat kota=1 | Kelas=0) = 0.0000
P(Jarak dai pusat kota=1 | Kelas=1) = 0.8000
P(Jarak dai pusat kota=2 | Kelas=0) = 0.4000
P(Jarak dai pusat kota=2 | Kelas=1) = 0.2000

Ada angkutan umum:
P(Ada angkutan umum=1 | Kelas=0) = 0.8000
P(Ada angkutan umum=1 | Kelas=1) = 0.4000
P(Ada angkutan umum=0 | Kelas=0) = 0.2000
P(Ada angkutan umum=0 | Kelas=1) = 0.6000


In [ ]:
import pandas as pd

# Fungsi untuk menghitung probabilitas posterior berdasarkan prior dan likelihood
def calculate_posterior(sample, prior_probs, feature_probs):
    posterior_probs = {}

    for class_value in prior_probs.index:
        # Inisialisasi probabilitas posterior dengan probabilitas prior
        posterior = prior_probs[class_value]

        # Kalikan dengan likelihood dari setiap fitur dalam sample
        for feature in sample.index:
            feature_value = sample[feature]

            # Menghitung likelihood untuk setiap fitur dan kelas
            if feature_value in feature_probs[feature]:
                likelihood = feature_probs[feature][feature_value].get(class_value, 0)
            else:
                likelihood = 0  # Jika fitur tidak ditemukan dalam kelas, beri nilai 0

            # Jika likelihood adalah 0, maka posterior menjadi 0
            if likelihood == 0:
                posterior = 0
                break  # Jika salah satu likelihood 0, posterior menjadi 0

            # Kalikan posterior dengan likelihood
            posterior *= likelihood

        posterior_probs[class_value] = posterior

    return posterior_probs

# Dataset dan probabilitas prior (contoh)
feature_probs = {
    'Harga Tanah': {
        0: {0: 0.2, 1: 0.6},  # Probabilitas untuk Harga Tanah = Mahal
        1: {0: 0.4, 1: 0.2},  # Probabilitas untuk Harga Tanah = Murah
        2: {0: 0.4, 1: 0.2}   # Probabilitas untuk Harga Tanah = Sedang
    },
    'Jarak dari pusat kota': {
        0: {0: 0.6, 1: 0.0},  # Probabilitas untuk Jarak = Dekat
        1: {0: 0.0, 1: 0.8},  # Probabilitas untuk Jarak = Jauh
        2: {0: 0.4, 1: 0.2}   # Probabilitas untuk Jarak = Sedang
    },
    'Ada angkutan umum': {
        0: {0: 0.2, 1: 0.6},  # Probabilitas untuk Ada angkutan umum = Tidak
        1: {0: 0.8, 1: 0.4}   # Probabilitas untuk Ada angkutan umum = Ada
    }
}

# Probabilitas prior (misalnya, berdasarkan data training set)
prior_probs = pd.Series({
    0: 0.5,  # Kelas Iya (0)
    1: 0.5   # Kelas Tidak (1)
})

# Tiga sampel data baru untuk prediksi
sample_data_list = [
    pd.Series({'Harga Tanah': 1, 'Jarak dari pusat kota': 1, 'Ada angkutan umum': 1}),  # Murah, Jauh, Tidak
    pd.Series({'Harga Tanah': 0, 'Jarak dari pusat kota': 0, 'Ada angkutan umum': 0}),  # Mahal, Dekat, Ada
    pd.Series({'Harga Tanah': 0, 'Jarak dari pusat kota': 1, 'Ada angkutan umum': 1})   # Mahal, Jauh, Tidak
]

# Menampilkan kelas yang lebih mudah dibaca
kelas_map = {0: "Iya", 1: "Tidak"}

# Prediksi untuk setiap sampel
for i, sample_data in enumerate(sample_data_list, 1):
    # Hitung probabilitas posterior untuk sample data baru
    posterior_probs = calculate_posterior(sample_data, prior_probs, feature_probs)

    # Menampilkan probabilitas posterior dengan format desimal
    print(f"\nPrediksi untuk Sampel {i}:")
    for class_value, posterior in posterior_probs.items():
        print(f"P(Kelas={kelas_map[class_value]} | Data) = {posterior:.4f}")  # Format 4 desimal

    # Menentukan prediksi kelas dengan probabilitas tertinggi
    predicted_class = max(posterior_probs, key=posterior_probs.get)
    print(f"Kelas yang diprediksi: {kelas_map[predicted_class]}")


Prediksi untuk Sampel 1:
P(Kelas=Iya | Data) = 0.0000
P(Kelas=Tidak | Data) = 0.0320
Kelas yang diprediksi: Tidak

Prediksi untuk Sampel 2:
P(Kelas=Iya | Data) = 0.0120
P(Kelas=Tidak | Data) = 0.0000
Kelas yang diprediksi: Iya

Prediksi untuk Sampel 3:
P(Kelas=Iya | Data) = 0.0000
P(Kelas=Tidak | Data) = 0.0960
Kelas yang diprediksi: Tidak


In [ ]:
from sklearn.model_selection import train_test_split

# Fungsi evaluasi dengan split data
def evaluate_with_split(X, y, train_size):
    # Membagi data menjadi training dan testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)

    # Menghitung prior probabilities untuk data training
    train_prior_probs = y_train.value_counts(normalize=True)

    # Menghitung likelihood probabilities untuk data training
    train_feature_probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    for feature in X_train.columns:
        for value in X_train[feature].unique():
            for class_value in y_train.unique():
                num = len(X_train[(X_train[feature] == value) & (y_train == class_value)])
                den = y_train.value_counts()[class_value]
                train_feature_probs[feature][value][class_value] = num / den if den != 0 else 0

    # Prediksi pada data testing
    def predict_row(row):
        posterior_probs = calculate_posterior(row, train_prior_probs, train_feature_probs)
        return max(posterior_probs, key=posterior_probs.get)

    y_pred = X_test.apply(predict_row, axis=1)

    # Evaluasi model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    return accuracy, report

# Split dengan 80% Training dan 20% Testing
X = encoded_df.drop(columns='Dipilih untuk perumahan')
y = encoded_df['Dipilih untuk perumahan']

results = {}

# Evaluasi dengan 80% Training, 20% Testing
accuracy_80, report_80 = evaluate_with_split(X, y, 0.8)
results["80% Training, 20% Testing"] = {"accuracy": accuracy_80, "report": report_80}

# Evaluasi dengan 70% Training, 30% Testing
accuracy_70, report_70 = evaluate_with_split(X, y, 0.7)
results["70% Training, 30% Testing"] = {"accuracy": accuracy_70, "report": report_70}

# Evaluasi dengan 60% Training, 40% Testing
accuracy_60, report_60 = evaluate_with_split(X, y, 0.6)
results["60% Training, 40% Testing"] = {"accuracy": accuracy_60, "report": report_60}

# Menampilkan hasil
for key, value in results.items():
    print(f"\n{key}:")
    print(f"Akurasi: {value['accuracy']:.2f}")
    print(f"Laporan Klasifikasi:")
    print(value['report'])


80% Training, 20% Testing:
Akurasi: 1.00
Laporan Klasifikasi:
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1.0}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1.0}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2.0}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2.0}}

70% Training, 30% Testing:
Akurasi: 0.67
Laporan Klasifikasi:
{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1.0}, '1': {'precision': 1.0, 'recall': 0.5, 'f1-score': 0.6666666666666666, 'support': 2.0}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.75, 'recall': 0.75, 'f1-score': 0.6666666666666666, 'support': 3.0}, 'weighted avg': {'precision': 0.8333333333333334, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 3.0}}

60% Training, 40% Testing:
Akurasi: 0.75
Laporan Klasifikasi:
{'0': {'precision': 0.6666666666666666, 'recal

## ***REMINDER FOR ME: MAKE SURE TO CLEAN CODE FOR FUTURE UNDERSTANDING***